CS524: Introduction to Optimization <br>Lecture 16: n-dimensional io
======================================
## by Michael C. Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison
## October 11, 2023


# Introduction
- The data does not always in the .gams file. Sometimes it is stored in other format, such as text file, Excel file, and GDX file
- In this lecture we will cover ways that GAMS reads and writes diifferent types of data. 

## `Put` statement
The `put` statement is at the core of the put writing facility.
It has two different functions:
- it specifies which of the previously defined put files is the current file
- it writes output to that file. 

The basic structure of the put writing facility in its simplest form is as follows:
    
    File file_name {,file_name};
    put file_name;
    put item {,item};

- [The Put Writing Facility](https://www.gams.com/latest/docs/UG_Put.html)  
- `put` statement allows execution time writing of file.

    _For example:_  
        file optfile /cplex.opt/;  
        put optfile;  
        put 'lpmethod 3'/;  
        put 'names no'/;  
        putclose;  

Here is another example of the `put` statement:

    file factors /factors.dat/, results /results.dat/ ;
    put factors ;
    put ’Transportation Model Factors’///
    ’Freight cost ’, f,
    @1#6, ’Plant capacity’/;
    loop(i, put @3, i.tl, @15, a(i)/);
    put /’Market demand’/;
    loop(j, put @3, j.tl, @15, b(j)/);
    put results;put ’Transportation Model Results’// ;
    loop((i,j), put i.tl, @12, j.tl, @24, x.l(i,j):8:4 /);    

After execution, the put file __factors.dat__ will look as follows:

Transportation Model　Factors

Freight cost　　　　　　90.00

Plant capacity<br>
　　Seattle　　　　　　　350.00<br>
　　San-Diego　　　　　 600.00<br>

Market demand<br>
　　New-York　　　　　　325.00<br>
　　Chicago　　　　　　 300.00<br>
　　Topeka　　　　　　　275.00<br>

<i>Notes:<i/>
- `/` is for carriage return
- `@` Move to column
- `#` Move to row
- `.tl` gives 'title' (string form) of index
- remember display?
    - x.l(i,j):8:4 will return level of x vars using field width 8,4 decimals

# `$onecho`, `$offecho

- [$ control options](https://www.gams.com/latest/docs/UG_DollarControlOptions.html)  
- `$onecho` and `$offecho` happen at compile time, so cannot update after a solve

    _For example:_  
    `$onecho` > cplex.opt  
    lpmethod 3  
    names no  
    `$offecho`  

## File input for GAMS

- [Data Exchange with Text Files](https://www.gams.com/latest/docs/UG_DataExchange_ASCII.html)
- The `include` compile-time command is used to instruct the GAMS compiler to include the context of a different file at the current position of the input stream.
- It works like a macro, just insert the contents of file at location
- Note that The include file (must) normally be in the current 'include directory' of GAMS
- Data files can easily be dumped from Excel


For example, if we want to create a table d in gams:

|           | new-york | chicago | topeka |
|-----------|----------|---------|--------|
| seattle   | 2.5      | 1.7     | 1.8    |
| san-diego | 2.5      | 1.8     | 1.4    |

We can use the `$include` statement followed by the filename to incorporate the data, instead of using the `set` statement.

Here the __data.inc__ should contain following content:

    d("seattle","new-york") = 2.5;
    d("san-diego","new-york") = 2.5;
    d("seattle","chicago") = 1.7;
    d("san-diego","chicago") = 1.8;
    d("seattle","topeka") = 1.8;
    d("san-diego","topeka") = 1.4;

In [1]:
%load_ext gams.magic
m = gams.exchange_container

ModuleNotFoundError: No module named 'gams'

In [2]:
%%gams
set i /seattle, san-diego/;
set j /new-york, chicago, topeka/;
Parameter d(i,j) 'distance in thousands of miles';
$include data.inc

UsageError: Cell magic `%%gams` not found.


In [3]:
# extract data back into python
d = m.data['d']
display(d.records,d.pivot())

,i,j,value
0,seattle,new-york,2.5
1,seattle,chicago,1.7
2,seattle,topeka,1.8
3,san-diego,new-york,2.5
4,san-diego,chicago,1.8
5,san-diego,topeka,1.4


,new-york,chicago,topeka
seattle,2.5,1.7,1.8
san-diego,2.5,1.8,1.4


In [4]:
%gams_cleanup --closedown

## GAMS include CSV 

- See notebooks:   
- [GAMS read csv file](./16gamsreadcsv.ipynb)  
- [GAMS write csv file](./16gamswritecsv.ipynb)  

## GAMS and GDX 

- [Reading and writing GDX files](https://www.gams.com/latest/docs/UG_GDX.html)
- See notebook:  
- [GDX read and write](./16gdx.ipynb)  